In [28]:
import os
import copy
import pandas as pd
import csv

for file in os.listdir(os.getcwd()):
    if file.endswith('intergenic_snpeff.vcf'):
        print "Parsing snpeff output for {}... \n".format(file)
        # create command to parse snpeff
        df = pd.read_csv(file, sep='\t', skiprows=3, usecols=['#CHROM', 'POS', 'ID', 'REF', 'ALT', 'INFO'])
        
        for row in df.itertuples():
            fixed_cols = list(row[:-1])
            parts = row[-1].split('|')
            
            # the first one is CLOSEST=0 or something
            fixed_cols.append(parts.pop(0))
            
            for el in parts:
                my_row = copy.deepcopy(fixed_cols)
                my_row.extend(el.split(','))
                
                out = ','.join(map(str, my_row))            

                with open('./test.csv','a') as outfile:
                    writer = csv.writer(outfile)
                    writer.writerow([out ])

Parsing snpeff output for chr8_test_vars_intergenic_verifiedintergenic_snpeff.vcf... 



In [ ]:
import subprocess

snpeff_oneperline_perl = os.path.join('D:\\', 'Documents', 'tools', 'snpEff', 'scripts', 'vcfEffOnePerLine_intergenic.pl')
java_path = 'java'
snpsift_jar = os.path.join('D:\\', 'Documents', 'tools', 'snpEff', 'SnpSift.jar')

for file in os.listdir(os.getcwd()):
    if file.endswith('intergenic_snpeff.vcf'):
        print "Parsing snpeff output for {}... \n".format(file)
        tsv_out = str('.'.join(file.split('.')[:-1]) if '.' in file else file) + '_parsed.tsv'
        # create command to parse snpeff
        snpout_cmd = 'cat {} | {} | {} -jar {} extractFields \
        - CHROM POS ID REF ALT "ANN[*].GENE" "ANN[*].GENEID" "ANN[*].EFFECT" "ANN[*].IMPACT" \
        "ANN[*].FEATURE" "ANN[*].FEATUREID" "ANN[*].BIOTYPE" "ANN[*].RANK" \
        "ANN[*].HGVS_C" "ANN[*].HGVS_P" > {}'.format(file, snpeff_oneperline_perl, \
        java_path, snpsift_jar,tsv_out)
        # call subprocess and communicate to pipe output between commands
        ps = subprocess.Popen(snpout_cmd,shell=True,stdout=subprocess.PIPE,stderr=subprocess.STDOUT)
        print ps.communicate()[0]